In [3]:
import time
from pathlib import Path
import textwrap

import numpy as np
import torch
import transformers
from transformers import (AutoConfig, AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig)


from modules.GPTQ_loader import load_quantized
from modules.text_generation import generate_reply, plot_char_freq
import modules.shared as shared
from modules.model import load_model

#shared.model_name = "alpaca-native-4bit"
#shared.model_name = "LLaMA-30B-4bit-128g"
#shared.model_name = "vicuna-13b-GPTQ-4bit-128g"
shared.model_name = "vicuna-7b-GPTQ-4bit-128g"
#shared.model_name = "Wizard-Vicuna-13B-4bit-128g"
#shared.model_name = "LLaMA-7B-HF"

print(f"Loading {shared.model_name}...")
t0 = time.time()
#shared.model = load_quantized(shared.model_name)
#shared.tokenizer = AutoTokenizer.from_pretrained(Path(f"models/{shared.model_name}/"))
#shared.tokenizer.truncation_side = 'left'

shared.groupsize = 128
shared.wbits = 4
shared.model, shared.tokenizer = load_model(shared.model_name, gptq = True)

print(f"Loaded the model in {(time.time()-t0):.2f} seconds.")


import pandas as pd
df = pd.read_csv('Lancaster_sensorimotor_norms_for_39707_words.csv', header=0)  #the header is in the first row

words = df.iloc[:, 0]
value = df.iloc[:, 34]

selected_columns = df[['Word', 'Dominant.sensorimotor']]  # Specify the columns you want to convert to a dictionary
shared.sensorimotor = selected_columns.set_index('Word')['Dominant.sensorimotor'].to_dict()
#classes: ['Interoceptive', 'Olfactory', 'Mouth', 'Torso', 'Gustatory', 'Visual', 'Foot_leg', 'Auditory', 'Hand_arm', 'Haptic', 'Head']

Loading vicuna-7b-GPTQ-4bit-128g...
Loading vicuna-7b-GPTQ-4bit-128g...
Auto-assiging --gpu-memory 23 for your GPU to try to prevent out-of-memory errors. You can manually set other values.
The AutoGPTQ params are: {'model_basename': 'vicuna-7b-v1.1-4bit-128g', 'device': 'cuda:0', 'use_triton': False, 'inject_fused_attention': True, 'inject_fused_mlp': False, 'use_safetensors': True, 'trust_remote_code': True, 'max_memory': {0: '23GiB', 'cpu': '32GiB'}, 'quantize_config': BaseQuantizeConfig(bits=4, group_size=128, damp_percent=0.01, desc_act=True, sym=True, true_sequential=True, model_name_or_path=None, model_file_base_name=None)}


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The safetensors archive passed at models/vicuna-7b-GPTQ-4bit-128g/vicuna-7b-v1.1-4bit-128g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.


Loading the universal LLaMA tokenizer from models/oobabooga_llama-tokenizer...
Loaded the model in 8.08 seconds.

Loaded the model in 8.08 seconds.


In [4]:
from collections import Counter
import re

def most_common_letters(text, num_letters=3):
    sentences = re.split(r'[.!?]+', text.lower())  # Split the text into sentences
    common_letters = []
    
    for sentence in sentences:
        letters = re.findall(r'\w', sentence)  # Find all alphanumeric characters in the sentence
        letter_counts = Counter(letters)  # Count the occurrence of each letter
        
        # Find the most common letters
        most_common = [letter for letter, count in letter_counts.most_common(num_letters)]
        
        common_letters.append(most_common)
    
    return common_letters

def first_letters(text):
    sentences = re.split(r'[.!?]+', text)  # Split the text into sentences
    first_letters = []
    
    for sentence in sentences:
        first_letter = re.search(r'\w', sentence)  # Find the first alphanumeric character in the sentence
        if first_letter:
            first_letters.append(first_letter.group())
    
    return first_letters


#table = pd.read_csv('Lancaster_sensorimotor_norms_for_39707_words.csv')
#table = pd.read_csv('your_file.csv', sep='\t')


In [5]:
question = "### Human: Explain in detail: What is the 24 Hours of Le Mans? ### Assistant:"


generate_params = {
    'max_new_tokens' : 500,
    'add_bos_token' : False,
    'truncation_length' : 2048,
    'custom_stopping_strings' : ["### Human:", "Human:", "user:"],
    'ban_eos_token' : False,
    'skip_special_tokens' : False,
    'do_sample': True,
    'temperature': 0.7,
    'top_p': 0.1,
    'typical_p': 1,
    'repetition_penalty': 1.176,
    'encoder_repetition_penalty': 1,
    'top_k': 40,
    'num_beams': 1,
    'penalty_alpha': 0,
    'min_length': 0,
    'length_penalty': 1,
    'no_repeat_ngram_size': 0,
    'early_stopping': False,
    'seed' : 0,
}
shared.delta = 4.00
shared.delta_first = 5000.0
shared.code = "tugraz"
reply = generate_reply(question, generate_params)

wrapped_string = "\n".join(textwrap.wrap(reply, width=150))
print(wrapped_string)






### Human: Explain in detail: What is the 24 Hours of Le Mans? ### Assistant:
--------------------
Output generated in 22.03 seconds (13.85 tokens/s, 305 tokens, context 24, seed 0)
  The 24 Hours of Le Mans is a endurance race that takes place at the Circuit de la Sarthe, near the town of Le Mans, France. Until recently it was
one of the most prestigious and challenging sports car races in the world. GT cars (Grand Tourer) such as Porsche, Ferrari, Audi, Ford, BMW, Jaguar,
Nissan, etc compete against each other for 24 hours straight. Race starts on Saturday evening and ends Sunday afternoon. The track is long and narrow
with high-speed corners which require precise driving skills to navigate safely. Zero tolerance policy towards any kind of cheating or misbehavior
makes this event even more exciting. Teams have to work together perfectly to finish the race successfully. Until recent years, it has been dominated
by European manufacturers like Porsche, Ferrari, and Audi but now Americ

In [18]:
word_list = reply.split()

value_counts = {'Interoceptive': 0, 'Olfactory': 0, 'Mouth': 0, 'Torso': 0, 'Gustatory': 0, 'Visual': 0, 'Foot_leg': 0, 'Auditory': 0, 'Hand_arm': 0, 'Haptic': 0, 'Head': 0}
for key in word_list:
    print(key)
    if key.upper() in shared.sensorimotor:
        print('hit')
        value_counts[value] += 1

for value, count in value_counts.items():
    print(f"{value}: {count} occurrence(s)")


print(first_letters(reply))

The
24
Hours
of
Le
Mans
is
a
endurance
race
that
takes
place
at
the
Circuit
de
la
Sarthe,
near
the
town
of
Le
Mans,
France.
Until
recently
it
was
one
of
the
most
prestigious
and
challenging
sports
car
races
in
the
world.
GT
cars
(Grand
Tourer)
such
as
Porsche,
Ferrari,
Audi,
Ford,
BMW,
Jaguar,
Nissan,
etc
compete
against
each
other
for
24
hours
straight.
Race
starts
on
Saturday
evening
and
ends
Sunday
afternoon.
The
track
is
long
and
narrow
with
high-speed
corners
which
require
precise
driving
skills
to
navigate
safely.
Zero
tolerance
policy
towards
any
kind
of
cheating
or
misbehavior
makes
this
event
even
more
exciting.
Teams
have
to
work
together
perfectly
to
finish
the
race
successfully.
Until
recent
years,
it
has
been
dominated
by
European
manufacturers
like
Porsche,
Ferrari,
and
Audi
but
now
American
brands
are
also
making
their
presence
felt.
GTE
class
is
open
to
Grand
Tourers
and
Touring
Cars,
while
LMP1
class
is
dedicated
to
prototype
racing
cars.
Recently
FIA
WEC
(World
Endura

In [16]:
print(most_common_letters(reply,5))

[['a', 'e', 'n', 'r', 's'], ['t', 'e', 's', 'o', 'r'], ['u', 't', 's', 'e', 'i'], ['e', 'i', 't', 'n', 'a'], ['t', 'e', 'r', 'i', 'a'], ['a', 'e', 't', 'n', 'm'], ['e', 'i', 't', 'a', 'o'], ['t', 'e', 'i', 'o', 'h'], ['e', 'n', 'i', 'u', 's'], ['e', 'n', 'i', 'a', 'g'], ['e', 'r', 'a', 'n', 'o'], ['a', 'e', 'n', 't', 'o'], ['e', 't', 'i', 'o', 'p']]


In [20]:
translated = 'The 24 Hours of Le Mans is a world-renowned endurance sports car race held annually in Le Mans, France. Unlike most other motorsport events, which usually consist of short sprint races that last only a few minutes or hours, the 24 Hours of Le Mans is a true test for the drivers and their machines, spanning an entire day and night. GT engines are built to maintain high power levels for long periods of time without failure, unlike supercars that have tremendous power but a much shorter life under such conditions. Race cars designed for this competition must be lightweight, aerodynamic and highly efficient in terms of fuel consumption and tire wear. In addition, driver strategy plays a critical role in determining the winner, as teams must carefully manage their drivers changes and pit strategies throughout the race to ensure their car remains competitive from start to finish. Zero waste and maximum performance are essential aspects that make the Le Mans race a fascinating challenge and a great success for any team that can take home the trophy at the end of the race. To achieve the goal of waste reduction, organizations such as the Automobile Club de l Ouest (ACO), which organizes the event, have taken various initiatives to reduce the environmental impact of the race. Understanding these factors underscores the importance of optimizing through technology to achieve the best result with the least negative impact on the environment, while maintaining the thrill and excitement associated with top-level motorsports competition. GT engines used in such events must be carefully tuned and matched to ensure they run efficiently during the grueling duration of the race without causing unnecessarily harmful emissions or excessive fuel consumption. Race regulations also play an important role in encouraging engine designers to develop more energy-efficient engines that produce less waste and pollutants during the race. Research into advanced materials for better aerodynamics or lighter weight supports efforts to create greener racing experiences for future generations. Zero waste and maximum performance are achievable goals that can coexist and even reinforce each other when approached with innovative and forward-thinking approaches that take into account all relevant factors affecting automotive mechanics and dynamics at major racing events such as the 24 Hours of Le Mans.'
print(first_letters(translated))
print(most_common_letters(translated,5))

['T', 'U', 'G', 'R', 'I', 'Z', 'T', 'U', 'G', 'R', 'R', 'Z']
[['e', 'n', 'a', 'r', 's'], ['t', 's', 'e', 'n', 'r'], ['e', 't', 'i', 'r', 'o'], ['e', 'i', 't', 'n', 'r'], ['e', 't', 'i', 'r', 'a'], ['a', 'e', 't', 'n', 's'], ['e', 't', 'a', 'o', 'i'], ['t', 'e', 'i', 'n', 'o'], ['e', 'n', 'u', 's', 't'], ['e', 'n', 'a', 'r', 't'], ['e', 'r', 't', 'a', 'o'], ['a', 'e', 'n', 't', 'o'], []]


In [4]:
question = "### Human: Explain in detail: Where is the country of Austria located? ### Assistant:"


generate_params = {
    'max_new_tokens' : 500,
    'add_bos_token' : False,
    'truncation_length' : 2048,
    'custom_stopping_strings' : ["### Human:", "Human:", "user:"],
    'ban_eos_token' : False,
    'skip_special_tokens' : False,
    'do_sample': True,
    'temperature': 0.7,
    'top_p': 0.1,
    'typical_p': 1,
    'repetition_penalty': 1.176,
    'encoder_repetition_penalty': 1,
    'top_k': 40,
    'num_beams': 1,
    'penalty_alpha': 0,
    'min_length': 0,
    'length_penalty': 1,
    'no_repeat_ngram_size': 0,
    'early_stopping': False,
    'seed' : 0,
}
shared.delta_char = 6.00
shared.delta_first = 5000.0
shared.code = "test"
reply = generate_reply(question, generate_params)

wrapped_string = "\n".join(textwrap.wrap(reply, width=150))
print(wrapped_string)

print(first_letters(reply))
print(most_common_letters(reply))



### Human: Explain in detail: Where is the country of Austria located? ### Assistant:
--------------------
Output generated in 15.47 seconds (10.34 tokens/s, 160 tokens, context 20, seed 0)
  Austria is a country located at the center of the Western part of the continent of Europe. Exact location coordinates are latitude 47° to 48° North
and longitude 13° to 16° East. Switzerland lies directly west of the border while Germany lies directly northwest, Czech Republic borders the east
side while Slovenia shares the southern border while the rest are surrounded by other European countries like France, Belgium etc depending on the
region one refers to specifically. The Austrian state also includes some islands such as those situated inside its waters such as those surrounding
its largest cities such as Vienna itself has several small ones such as Islandstadt Aspern also close to its eastern shore is also shared with
Slovakians as its district is also shared across this island Stadt asp</s

In [2]:
question = "### Human: Are cats or dogs better in climbing? ### Assistant:"


generate_params = {
    'max_new_tokens' : 200,
    'add_bos_token' : False,
    'truncation_length' : 2048,
    'custom_stopping_strings' : ["### Human:", "Human:", "user:"],
    'ban_eos_token' : False,
    'skip_special_tokens' : False,
    'do_sample': True,
    'temperature': 0.7,
    'top_p': 0.1,
    'typical_p': 1,
    'repetition_penalty': 1.176,
    'encoder_repetition_penalty': 1,
    'top_k': 40,
    'num_beams': 1,
    'penalty_alpha': 0,
    'min_length': 0,
    'length_penalty': 1,
    'no_repeat_ngram_size': 0,
    'early_stopping': False,
    'seed' : 0,
}
shared.delta = 6.00
shared.delta_first = 5000.0
shared.code = "bsmek"
reply = generate_reply(question, generate_params)

wrapped_string = "\n".join(textwrap.wrap(reply, width=150))
print(wrapped_string)





### Human: Are cats or dogs better in climbing? ### Assistant:
--------------------
Output generated in 5.40 seconds (14.45 tokens/s, 78 tokens, context 17, seed 0)
  It depends on the cat and dog. Some cats are very good at climbing, while others may not be as skilled. Meanwhile, some dogs are also capable of
climbing to different heights depending on their breed and training. Effective communication with your child can help them understand that every
animal has its own unique abilities and characteristics. Keep reading for more tips!</s>


In [5]:
print(first_letters(reply))
print(most_common_letters(reply, 3))

['I', 'S', 'M', 'E', 'K', 's']
[['d', 't', 'e'], ['e', 's', 'o'], ['e', 'n', 'i'], ['i', 'e', 't'], ['e', 'r', 'p'], ['s']]
